In [11]:
import pandas as pd

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras #케라스
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score

- 상병기호 : ['E10', 'I10', 'J00', 'J20', 'M50']

In [2]:
df = pd.read_csv('./data/hackathon.csv')
df = df.iloc[:, 1:]
df.sample(15)

,시도코드,성별코드,bmi,map,식전혈당(공복혈당),총 콜레스테롤,트리글리세라이드,혈색소,요단백,혈청크레아티닌,ast_alt2,감마 지티피,상병기호
131245,41,1,31.020408,90.333333,97.0,167.0,265.0,15.0,1.0,0.70,0,40.0,1
318003,11,1,25.711662,82.666667,79.0,256.0,165.0,17.7,1.0,1.40,0,61.0,2
141568,43,1,24.444444,99.000000,83.0,241.0,327.0,17.0,1.0,0.13,0,40.0,3
85373,29,2,25.390625,86.666667,97.0,224.0,114.0,12.8,1.0,0.60,0,17.0,0
35895,47,2,20.761246,96.666667,103.0,154.0,44.0,10.6,1.0,0.80,0,12.0,4
312178,11,2,22.892820,87.333333,105.0,137.0,111.0,12.7,3.0,0.50,0,29.0,2
153419,47,1,27.343750,98.666667,121.0,145.0,122.0,13.4,1.0,0.80,0,42.0,1
148058,11,1,23.875115,91.666667,99.0,158.0,198.0,15.8,1.0,1.00,0,26.0,4
131861,47,2,21.947874,90.000000,125.0,191.0,64.0,11.4,1.0,0.60,0,14.0,3
200629,11,1,24.930748,81.666667,117.0,183.0,124.0,15.4,1.0,1.10,0,15.0,2


In [3]:
from sklearn.preprocessing import LabelEncoder
# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

# 데이터에 LabelEncoder 적용
encoded_data = label_encoder.fit_transform(df['시도코드'])
df['시도코드'] = encoded_data

# 전처리

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330958 entries, 0 to 330957
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   시도코드        330958 non-null  int64  
 1   성별코드        330958 non-null  int64  
 2   bmi         330958 non-null  float64
 3   map         330958 non-null  float64
 4   식전혈당(공복혈당)  330958 non-null  float64
 5   총 콜레스테롤     330958 non-null  float64
 6   트리글리세라이드    330958 non-null  float64
 7   혈색소         330958 non-null  float64
 8   요단백         330958 non-null  float64
 9   혈청크레아티닌     330958 non-null  float64
 10  ast_alt2    330958 non-null  int64  
 11  감마 지티피      330958 non-null  float64
 12  상병기호        330958 non-null  int64  
dtypes: float64(9), int64(4)
memory usage: 32.8 MB


In [4]:
temp = df.select_dtypes(include=['int64']).astype('category')

temp2 = df.select_dtypes(include=['float64']).astype('float32')

scaler = StandardScaler()
temp2 = pd.DataFrame(scaler.fit_transform(temp2), columns=temp2.columns) #np -> pd

df2 = pd.concat([temp2, temp], axis=1)
df2.info()
display(df2[:10])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330958 entries, 0 to 330957
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   bmi         330958 non-null  float32 
 1   map         330958 non-null  float32 
 2   식전혈당(공복혈당)  330958 non-null  float32 
 3   총 콜레스테롤     330958 non-null  float32 
 4   트리글리세라이드    330958 non-null  float32 
 5   혈색소         330958 non-null  float32 
 6   요단백         330958 non-null  float32 
 7   혈청크레아티닌     330958 non-null  float32 
 8   감마 지티피      330958 non-null  float32 
 9   시도코드        330958 non-null  category
 10  성별코드        330958 non-null  category
 11  ast_alt2    330958 non-null  category
 12  상병기호        330958 non-null  category
dtypes: category(4), float32(9)
memory usage: 12.6 MB


,bmi,map,식전혈당(공복혈당),총 콜레스테롤,트리글리세라이드,혈색소,요단백,혈청크레아티닌,감마 지티피,시도코드,성별코드,ast_alt2,상병기호
0,0.182520,-0.621876,-0.717914,0.831830,0.499025,1.426068,-0.253558,0.764726,-0.219147,2,1,0,1
1,1.457176,-0.621876,5.794160,-1.861579,1.328992,0.977745,-0.253558,-0.443400,-0.044303,11,1,0,2
2,1.457176,-0.621876,5.794160,-1.861579,1.328992,0.977745,-0.253558,-0.443400,-0.044303,11,1,0,3
3,-0.549760,-0.373717,-0.003174,-0.503164,-0.735804,-0.559361,-0.253558,-0.443400,0.033405,8,2,0,2
4,-0.361858,-0.311677,-0.479667,-1.088688,0.418053,-0.751499,-0.253558,-0.443400,-0.491125,8,2,0,0
5,0.338024,1.115238,-0.400252,0.574200,1.966650,1.554161,-0.253558,0.462694,-0.258001,5,1,0,3
6,0.338024,1.115238,-0.400252,0.574200,1.966650,1.554161,-0.253558,0.462694,-0.258001,5,1,0,2
7,1.219763,2.014815,0.115949,4.087343,1.086075,0.081100,-0.253558,-0.443400,0.013978,11,2,0,3
8,1.219763,2.014815,0.115949,4.087343,1.086075,0.081100,-0.253558,-0.443400,0.013978,11,2,0,2
9,-0.944999,-0.559836,-0.400252,1.300249,-0.492887,-1.520052,-0.253558,-0.141369,-0.491125,8,2,0,0


# deep

In [29]:
train_input, test_input, train_target, test_target = train_test_split(df2.iloc[:, :-1], df2.iloc[:,-1], test_size=0.2, random_state=42)

In [30]:
# 모델 생성
model = Sequential()

# 입력층과 은닉층 추가
model.add(Dense(128, activation='relu', input_shape=(train_input.shape[1],)))  # input_dim은 입력 특성의 차원
model.add(Dropout(0.5))  # 드롭아웃을 통한 과적합 방지
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

# 출력층 추가 (클래스 수에 맞게 노드 수 설정)
num_classes = 5
model.add(Dense(num_classes, activation='softmax'))  # num_classes는 클래스의 개수

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [31]:
# 타겟값을 one-hot encoding 형태로 변환
train_target_encoded = to_categorical(train_target)
test_target_encoded = to_categorical(test_target)

checkpoint_cb = keras.callbacks.ModelCheckpoint('./best_model.h5', save_best_only=True)
earlystop_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

In [33]:
history = model.fit(train_input, train_target_encoded, epochs=200,
                      validation_data=(test_input, test_target_encoded),
                      callbacks=[checkpoint_cb, earlystop_cb])
score = model.evaluate(test_input, test_target_encoded)
score

Epoch 1/200
8274/8274 [==============================] - 10s 1ms/step - loss: 1.5227 - accuracy: 0.3199 - val_loss: 1.5235 - val_accuracy: 0.3206
Epoch 2/200
8274/8274 [==============================] - 11s 1ms/step - loss: 1.5226 - accuracy: 0.3199 - val_loss: 1.5233 - val_accuracy: 0.3206
Epoch 3/200
8274/8274 [==============================] - 11s 1ms/step - loss: 1.5226 - accuracy: 0.3199 - val_loss: 1.5234 - val_accuracy: 0.3206
Epoch 4/200
2069/2069 [==============================] - 2s 899us/step - loss: 1.5233 - accuracy: 0.3206


[1.5233477354049683, 0.3206278681755066]

-------

In [36]:
# 모델 생성
model = Sequential()

# 입력층과 은닉층 추가
model.add(Dense(128, activation='relu', input_shape=(train_input.shape[1],)))  # input_dim은 입력 특성의 차원
model.add(Dropout(0.3))  # 드롭아웃을 통한 과적합 방지
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.3))

# 출력층 추가 (클래스 수에 맞게 노드 수 설정)
num_classes = 5
model.add(Dense(num_classes, activation='softmax'))  # num_classes는 클래스의 개수

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [37]:
history = model.fit(train_input, train_target_encoded, epochs=200,
                      validation_data=(test_input, test_target_encoded),
                      callbacks=[checkpoint_cb, earlystop_cb])
score = model.evaluate(test_input, test_target_encoded)
score

Epoch 1/200
8274/8274 [==============================] - 12s 1ms/step - loss: 1.5267 - accuracy: 0.3177 - val_loss: 1.5235 - val_accuracy: 0.3206
Epoch 2/200
8274/8274 [==============================] - 13s 2ms/step - loss: 1.5227 - accuracy: 0.3198 - val_loss: 1.5235 - val_accuracy: 0.3206
Epoch 3/200
8274/8274 [==============================] - 13s 2ms/step - loss: 1.5226 - accuracy: 0.3199 - val_loss: 1.5233 - val_accuracy: 0.3206
Epoch 4/200
8274/8274 [==============================] - 13s 2ms/step - loss: 1.5226 - accuracy: 0.3199 - val_loss: 1.5234 - val_accuracy: 0.3206
Epoch 5/200
2069/2069 [==============================] - 2s 836us/step - loss: 1.5233 - accuracy: 0.3206


[1.5233367681503296, 0.3206278681755066]

# 튜닝_RFE

In [53]:
model = XGBClassifier(
    objective='multi:softmax',  # 다중분류
    num_class=5,  # 클래스 개수 설정
    enable_categorical=True, # 범주형 변수 유지
    max_depth = 5
)

# RFE를 사용하여 변수 선택
rfe = RFE(model, n_features_to_select=9)  # 선택할 특성의 개수를 지정합니다.

# 변수 선택 및 학습용 데이터 변환
X_train_rfe = rfe.fit_transform(train_input, train_target)
X_test_rfe = rfe.transform(test_input)

# 모델 학습
model.fit(X_train_rfe, train_target)

# 예측 수행
y_pred = model.predict(X_test_rfe)

# 정확도 계산
accuracy = accuracy_score(test_target, y_pred)
print(f'정확도: {accuracy}')

정확도: 0.31222806381435825


In [39]:
confusion_matrix(test_target, y_pred)

array([[    1,    24,  5351,   286,    27],
       [   13,     7,  9927,   586,    50],
       [    9,    64, 19554,  1493,   103],
       [    9,    40, 16409,   637,    51],
       [   18,    47, 10874,   591,    21]], dtype=int64)

In [36]:
df['상병기호'].value_counts()/len(df)

2    0.320047
3    0.260819
4    0.175651
1    0.157603
0    0.085881
Name: 상병기호, dtype: float64